In [1]:
from ConectionSQL import SQL
import pandas as pd
import glob
import os

In [2]:
sql=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=sql.ConexaoSQL()

In [3]:
querys={

    'Estoque':"""
        
    DECLARE @DTATUAL DATETIME

    SET @DTATUAL=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101);

    WITH TabSTK (cd_emp,local,cd_prod,qtde,qtde_pend_pedv,qtde_saldo,dt_ult_movimentacao) AS (

    SELECT estoque.cd_emp,local.descricao AS local,
    cd_prod,qtde,qtde_pend_pedv,(qtde-qtde_pend_pedv) as qtde_saldo,
    CONVERT(DATETIME,CAST(dt_ult_movimentacao AS DATE),101)
    FROM (
    SELECT cd_emp,cd_local,cd_prod,qtde,qtde_pend_pedv,
    dt_ult_movimentacao
    FROM estoque)estoque
    INNER JOIN local ON estoque.cd_local=local.cd_local AND estoque.cd_emp=local.cd_emp AND local.central=1
    INNER JOIN empresa ON estoque.cd_emp=empresa.cd_emp AND empresa.ativo=1),

    TabSTKDia (data,local,cd_emp,cd_prod,qtde,qtde_pend_pedv,qtde_saldo,dt_ultima) AS (

    SELECT estoque.data,local,estoque.cd_emp,estoque.cd_prod,estoque.qtde,estoque.qtde_pend_pedv,
    qtde_saldo,
    (SELECT MAX(est_dia.data) FROM est_dia WHERE estoque.cd_emp=est_dia.cd_emp AND estoque.cd_prod=est_dia.cd_prod
    AND est_dia.data<estoque.data) AS dt_ultima
    FROM (
    SELECT CONVERT(DATETIME,CAST(data AS DATE),101) AS data,local.descricao AS local,
    est_dia.cd_emp,cd_prod,SUM(qtde) AS qtde,SUM(qtde_pend_pedv) AS qtde_pend_pedv,
    SUM(qtde-qtde_pend_pedv) as qtde_saldo
    FROM est_dia
    INNER JOIN local ON est_dia.cd_local=local.cd_local AND est_dia.cd_emp=local.cd_emp AND local.central=1
    INNER JOIN empresa ON est_dia.cd_emp=empresa.cd_emp AND empresa.ativo=1
    --WHERE cd_prod=3
    GROUP BY data,est_dia.cd_emp,cd_prod,local.descricao)estoque),

    TabMovimentacao (data,local,cd_emp,cd_prod,qtde,qtde_pend_pedv,qtde_saldo,dt_ultima,penul_saldo) AS (

    SELECT Tab01.data,Tab01.local,Tab01.cd_emp,Tab01.cd_prod,Tab01.qtde,Tab01.qtde_pend_pedv,
    Tab01.qtde_saldo,Tab01.dt_ultima,Tab02.qtde_saldo AS penul_saldo
    FROM TabSTKDia AS Tab01
    INNER JOIN TabSTKDia AS Tab02 ON Tab01.cd_emp=Tab02.cd_emp AND Tab01.cd_prod=Tab02.cd_prod
    AND Tab01.dt_ultima=Tab02.data
    WHERE Tab01.data=@DTATUAL
    ),

    TabLinha (cd_linha,linha,categoria,secao,departamento) AS (

    SELECT cd_linha,linha.descricao as linha,categprd.descricao as categoria,
    secao.descricao as secao,depto.descricao as departamento
    FROM linha
    INNER JOIN categprd ON linha.cd_categprd=categprd.cd_categprd
    INNER JOIN secao ON linha.cd_secao=secao.cd_secao
    INNER JOIN depto ON secao.cd_depto=depto.cd_depto
    WHERE linha.ativo=1)


    SELECT TabSTK.local AS 'Local de Estoque',
    preco.cd_prod AS 'SKU',produto.descricao AS 'Produto',fabric.descricao AS 'Fabricante',
    TabLinha.departamento AS 'Departamento',TabLinha.secao AS 'Seção',
    TabLinha.categoria AS 'Categoria',TabLinha.linha AS 'Linha',
    TabSTK.qtde_saldo AS 'Saldo Atual',TabSTK.dt_ult_movimentacao AS 'Data da Movimentação',
    TabMovimentacao.penul_saldo AS 'Penúltima Movimentação',
    TabMovimentacao.dt_ultima AS 'Penúltima Data',
    CASE WHEN TabSTK.qtde_saldo<=1 THEN 'INATIVAR'
    WHEN TabSTK.qtde_saldo>0 AND TabMovimentacao.penul_saldo<=1 THEN 'REATIVAR'
    ELSE 'OK' END AS 'Acompanhamento'
    FROM preco
    INNER JOIN tab_pre ON preco.cd_tabela=tab_pre.cd_tabela
    INNER JOIN TabSTK ON preco.cd_prod=TabSTK.cd_prod
    INNER JOIN TabMovimentacao ON preco.cd_prod=TabMovimentacao.cd_prod
    INNER JOIN produto ON preco.cd_prod=produto.cd_prod and produto.ativo=1
    INNER JOIN TabLinha ON produto.cd_linha=TabLinha.cd_linha
    INNER JOIN fabric ON produto.cd_fabric=fabric.cd_fabric
    WHERE tab_pre.descricao LIKE '%CONSUMIDOR%' AND vl_preco>0
    and NOT TabLinha.departamento LIKE '%EQUIP%'
    
    """
    
}

In [4]:
tabelas_df=dict()

for tabela,query in querys.items():
    
    tabelas_df[tabela]=pd.read_sql(query,conectando)
    
    pass

In [5]:
tabelas_df['Estoque'].loc[tabelas_df['Estoque']['Acompanhamento']!='OK']

,Local de Estoque,SKU,Produto,Fabricante,Departamento,Seção,Categoria,Linha,Saldo Atual,Data da Movimentação,Penúltima Movimentação,Penúltima Data,Acompanhamento
0,DEMARCHI CENTRAL,2112,(200G) POLPINHA DE KIWI,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0,2022-04-08,0.0,2022-06-02,INATIVAR
2,DEMARCHI CENTRAL,9398,"CONGELADO JARDINEIRA S/ MILHO (1,020 KG)",DE MARCHI,CONGELADOS,VEGETAIS CONGELADOS,MIX,"LEGUMES 1,020",0.0,2022-06-03,240.0,2022-06-02,INATIVAR
4,DEMARCHI CENTRAL,2074,(200G) POLPINHA DE COCO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0,2021-10-29,0.0,2022-06-02,INATIVAR
15,DEMARCHI CENTRAL,2670,(200G) POLPINHA MIST KIW ABAC MAÇ HORT,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0,2021-10-29,0.0,2022-06-02,INATIVAR
19,DEMARCHI CENTRAL,1129,(200G) POLPINHA DE VITAMINA,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0,2021-01-27,0.0,2022-06-02,INATIVAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,DEMARCHI CENTRAL,1048,"CONGELADO BATATA MCCAIN CANOA NAC 6X2,5",MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA ESPECIALIDADE,BATATA PALITO,0.0,2022-05-28,0.0,2022-06-02,INATIVAR
347,DEMARCHI CENTRAL,2121,(200G) POLPINHA DE LIMAO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0,2021-10-18,0.0,2022-06-02,INATIVAR
349,DEMARCHI CENTRAL,8,POLPINHA DE ACAI (100G),DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,0.0,2021-10-18,0.0,2022-06-02,INATIVAR
352,DEMARCHI CENTRAL,547,SUCO DE MARACUJA 5 LITROS,DE MARCHI,RESFRIADOS,SUCOS,CONCENTRADO,CONCENTRADO ATÉ 5L,0.0,2021-10-18,0.0,2022-06-02,INATIVAR


In [6]:
tabelas_df['Estoque'].columns

Index(['Local de Estoque', 'SKU', 'Produto', 'Fabricante', 'Departamento',
       'Seção', 'Categoria', 'Linha', 'Saldo Atual', 'Data da Movimentação',
       'Penúltima Movimentação', 'Penúltima Data', 'Acompanhamento'],
      dtype='object')

In [7]:
#tabelas_df['Estoque'].drop(columns=['Acompanhamento'],inplace=True)

In [8]:
tabelas_df['Estoque']

,Local de Estoque,SKU,Produto,Fabricante,Departamento,Seção,Categoria,Linha,Saldo Atual,Data da Movimentação,Penúltima Movimentação,Penúltima Data,Acompanhamento
0,DEMARCHI CENTRAL,2112,(200G) POLPINHA DE KIWI,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0000,2022-04-08,0.0000,2022-06-02,INATIVAR
1,DEMARCHI CENTRAL,901,CONGELADO CENOURA BABY 300G DE MARCHI,FRAGOLE,CONGELADOS,VEGETAIS CONGELADOS,CENOURA BABY,LEGUMES 300G,127.0000,2022-06-03,147.0000,2022-06-02,OK
2,DEMARCHI CENTRAL,9398,"CONGELADO JARDINEIRA S/ MILHO (1,020 KG)",DE MARCHI,CONGELADOS,VEGETAIS CONGELADOS,MIX,"LEGUMES 1,020",0.0000,2022-06-03,240.0000,2022-06-02,INATIVAR
3,DEMARCHI CENTRAL,9543,"VERO SORBETTO CHOCOLATE 1,2KG/1,5L",DE MARCHI,CONGELADOS,SORBET,VERO SORBETTO,VERO SORBETTO,39.0000,2022-06-03,40.0000,2022-06-02,OK
4,DEMARCHI CENTRAL,2074,(200G) POLPINHA DE COCO,DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 200G,0.0000,2021-10-29,0.0000,2022-06-02,INATIVAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,DEMARCHI CENTRAL,756,CONGELADO BROCOLIS FRAGOLE (2KG),FRAGOLE,CONGELADOS,VEGETAIS CONGELADOS,BRÓCOLIS,LEGUMES 2KG,2.0000,2022-06-02,2.0000,2022-06-02,OK
358,DEMARCHI CENTRAL,96,POLPINHA DE FRAMBOESA (100G),DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,261.0000,2022-06-03,283.0000,2022-06-02,OK
359,DEMARCHI CENTRAL,611,"IQF MARACUJA C/SEM (1,020KG)",DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPÃO,"POLPÃO 1,020KG",328.3312,2022-06-03,346.3312,2022-06-02,OK
360,DEMARCHI CENTRAL,960,ACAI SPORT DM BALDE 910G MORANGO,DE MARCHI,CONGELADOS,AÇAÍ,SPORT,AÇAÍ SPORT,765.0000,2022-06-03,798.0000,2022-06-02,OK
